# Kepler Experiment Management - Jupyter Notebook Example

This notebook demonstrates how to use Kepler for experiment tracking and management in a Jupyter notebook environment. Kepler provides tools for defining experiment configurations, saving results, and monitoring progress.

## Setup

First, let's import the necessary libraries:

In [ ]:
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Import Kepler
from kepler import experiment, save_dataframe, save_dict

## Generate Synthetic Data

Let's create a synthetic classification dataset for our experiments:

In [ ]:
# Generate a synthetic classification dataset
X, y = make_classification(
    n_samples=1000,
    n_features=20,
    n_informative=10,
    n_redundant=5,
    random_state=42
)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")

## Define Experiment Configuration

Now, let's define the configuration for our experiment:

In [ ]:
# Define experiment configuration
config = {
    "model": "RandomForest",
    "n_estimators": 100,
    "max_depth": 10,
    "min_samples_split": 2,
    "min_samples_leaf": 1,
    "random_state": 42
}

## Run the Experiment

Now we'll run our experiment using Kepler's experiment context manager:

In [ ]:
# Run the experiment
with experiment("RandomForest-Notebook", config=config, tags=["notebook", "classification"]) as exp:
    exp.log_info("Starting RandomForest training experiment")
    
    # Initialize the model with our configuration
    model = RandomForestClassifier(
        n_estimators=config["n_estimators"],
        max_depth=config["max_depth"],
        min_samples_split=config["min_samples_split"],
        min_samples_leaf=config["min_samples_leaf"],
        random_state=config["random_state"]
    )
    
    # Log the start of training
    exp.log_info("Training model...")
    exp.update_progress(0, 3, "Starting model training")
    
    # Train the model
    start_time = time.time()
    model.fit(X_train, y_train)
    training_time = time.time() - start_time
    
    # Log training completion
    exp.log_info(f"Model training completed in {training_time:.2f} seconds")
    exp.update_progress(1, 3, "Model training completed")
    exp.set_metric("training_time", training_time)
    
    # Make predictions
    exp.log_info("Making predictions on test data...")
    y_pred = model.predict(X_test)
    exp.update_progress(2, 3, "Predictions generated")
    
    # Calculate metrics
    exp.log_info("Calculating performance metrics...")
    metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred),
        "recall": recall_score(y_test, y_pred),
        "f1_score": f1_score(y_test, y_pred)
    }
    
    # Log all metrics
    for metric_name, metric_value in metrics.items():
        exp.set_metric(metric_name, metric_value)
    
    # Log feature importances
    feature_importances = model.feature_importances_
    feature_importance_df = pd.DataFrame({
        'feature': [f'feature_{i}' for i in range(X.shape[1])],
        'importance': feature_importances
    }).sort_values('importance', ascending=False)
    
    # Save feature importances
    fi_path = save_dataframe(feature_importance_df, "feature_importances", exp.id)
    exp.log_info(f"Saved feature importances to {fi_path}")
    
    # Save metrics
    metrics_path = save_dict(metrics, "performance_metrics", exp.id)
    exp.log_info(f"Saved performance metrics to {metrics_path}")
    
    # Complete the experiment
    exp.update_progress(3, 3, "Experiment completed")
    exp.log_info("Experiment completed successfully")
    
    # Display results
    print(f"Experiment ID: {exp.id}")
    print("Performance Metrics:")
    for metric_name, metric_value in metrics.items():
        print(f"  {metric_name}: {metric_value:.4f}")

## Visualize Results

Now let's visualize some of the results from our experiment:

In [ ]:
# Plot feature importances
plt.figure(figsize=(10, 6))
plt.barh(feature_importance_df['feature'][:10], feature_importance_df['importance'][:10])
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Top 10 Feature Importances')
plt.tight_layout()
plt.show()

In [ ]:
# Plot metrics
plt.figure(figsize=(8, 6))
plt.bar(metrics.keys(), metrics.values())
plt.ylim(0, 1)
plt.title('Model Performance Metrics')
plt.tight_layout()
plt.show()

## Running Multiple Experiments

Let's run multiple experiments with different hyperparameters to compare their performance:

In [ ]:
# Define hyperparameter grid
param_grid = [
    {"n_estimators": 50, "max_depth": 5},
    {"n_estimators": 100, "max_depth": 10},
    {"n_estimators": 200, "max_depth": 15}
]

# Store results for comparison
experiment_results = []

# Run experiments for each parameter combination
for i, params in enumerate(param_grid):
    # Update the configuration
    config = {
        "model": "RandomForest",
        "n_estimators": params["n_estimators"],
        "max_depth": params["max_depth"],
        "min_samples_split": 2,
        "min_samples_leaf": 1,
        "random_state": 42
    }
    
    # Create experiment name
    exp_name = f"RF-NE{params['n_estimators']}-MD{params['max_depth']}"
    
    # Run the experiment
    with experiment(exp_name, config=config, tags=["notebook", "hyperparameter_tuning"]) as exp:
        exp.log_info(f"Starting experiment {i+1}/{len(param_grid)}")
        
        # Initialize and train the model
        model = RandomForestClassifier(
            n_estimators=config["n_estimators"],
            max_depth=config["max_depth"],
            min_samples_split=config["min_samples_split"],
            min_samples_leaf=config["min_samples_leaf"],
            random_state=config["random_state"]
        )
        
        # Train the model
        start_time = time.time()
        model.fit(X_train, y_train)
        training_time = time.time() - start_time
        exp.set_metric("training_time", training_time)
        
        # Make predictions and calculate metrics
        y_pred = model.predict(X_test)
        metrics = {
            "accuracy": accuracy_score(y_test, y_pred),
            "precision": precision_score(y_test, y_pred),
            "recall": recall_score(y_test, y_pred),
            "f1_score": f1_score(y_test, y_pred)
        }
        
        # Log metrics
        for metric_name, metric_value in metrics.items():
            exp.set_metric(metric_name, metric_value)
        
        # Save metrics
        save_dict(metrics, "performance_metrics", exp.id)
        
        # Store results for comparison
        result = {
            "experiment_id": exp.id,
            "experiment_name": exp_name,
            "n_estimators": params["n_estimators"],
            "max_depth": params["max_depth"],
            "training_time": training_time,
            **metrics
        }
        experiment_results.append(result)
        
        print(f"Completed experiment: {exp_name} (ID: {exp.id})")
        print(f"Accuracy: {metrics['accuracy']:.4f}, Training time: {training_time:.2f}s")
        print("-" * 50)

## Compare Experiment Results

Let's create a DataFrame to compare the results of our experiments:

In [ ]:
# Create a DataFrame with the results
results_df = pd.DataFrame(experiment_results)
results_df

In [ ]:
# Visualize the comparison of accuracy across experiments
plt.figure(figsize=(10, 6))
plt.bar(results_df['experiment_name'], results_df['accuracy'])
plt.ylim(0.8, 1.0)  # Adjust as needed
plt.ylabel('Accuracy')
plt.title('Accuracy Comparison Across Experiments')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Visualize the trade-off between accuracy and training time
plt.figure(figsize=(10, 6))
plt.scatter(results_df['training_time'], results_df['accuracy'], s=100)

# Add labels to each point
for i, row in results_df.iterrows():
    plt.annotate(row['experiment_name'], 
                 (row['training_time'], row['accuracy']),
                 textcoords="offset points", 
                 xytext=(0,10), 
                 ha='center')

plt.xlabel('Training Time (seconds)')
plt.ylabel('Accuracy')
plt.title('Accuracy vs. Training Time')
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

## Conclusion

In this notebook, we've demonstrated how to use Kepler for experiment tracking in a Jupyter notebook environment. We've shown how to:

1. Define experiment configurations
2. Track experiment progress and log metrics
3. Save experiment artifacts (DataFrames and dictionaries)
4. Run multiple experiments with different hyperparameters
5. Compare and visualize experiment results

Kepler makes it easy to keep track of your experiments, their configurations, and results, which is essential for reproducible data science workflows.

## Next Steps

You can view your experiments using the Kepler terminal UI by running the following command in a terminal:

```bash
kepler tui
```

Or list all experiments with:

```bash
kepler list
```

For more information about a specific experiment, use:

```bash
kepler info <experiment_id>
```